In [132]:
import os
import json
import numpy as np
import cv2
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras import layers, models

# Diretório onde estão armazenadas as imagens sem ser separadas por pastas
image_dir = r'C:\Users\bruna\OneDrive - Universidade do Minho\Tese Mestrado em Bioinformática\AGAR_dataset\AGAR_dataset\dataset'


In [133]:
# Função para carregar e redimensionar a imagem
def load_image(image_path, target_size=(224, 224)):
    try:
        image = Image.open(image_path)
        image = image.resize(target_size)  # Redimensionar a imagem para o tamanho desejado
        return np.array(image) / 255.0  # Normalizar a imagem
    except Exception as e:
        print(f"Erro ao carregar a imagem {image_path}: {e}")
        return None

In [134]:
# Carregar IDs do grupo de treinamento de um arquivo de texto
train_ids_file = r'C:\Users\bruna\OneDrive - Universidade do Minho\Tese Mestrado em Bioinformática\AGAR_dataset\AGAR_dataset\training_lists\lower_resolution_train.txt'
with open(train_ids_file, 'r') as file:
    train_ids = [str(id) for id in json.loads(file.read())]

In [135]:
# Carregar IDs do grupo de validação de um arquivo de texto
val_ids_file = r'C:\Users\bruna\OneDrive - Universidade do Minho\Tese Mestrado em Bioinformática\AGAR_dataset\AGAR_dataset\training_lists\lower_resolution_val.txt'
with open(val_ids_file, 'r') as file:
    val_ids = [str(id) for id in json.loads(file.read())]

In [136]:
# Verificar se os IDs foram carregados corretamente
print(f"IDs de treinamento carregados: {train_ids[:5]} ... ({len(train_ids)} no total)")
print(f"IDs de validação carregados: {val_ids[:5]} ... ({len(val_ids)} no total)")

IDs de treinamento carregados: ['16078', '16831', '16073', '16072', '16830'] ... (3319 no total)
IDs de validação carregados: ['14175', '14176', '15540', '14172', '14678'] ... (1107 no total)


In [137]:
# Lista para armazenar caminho da imagem e anotações correspondentes
train_data = []
val_data = []

# Iterar sobre todos os arquivos no diretório
for filename in os.listdir(image_dir):
    if filename.endswith('.json'):
        # Construir o caminho completo do arquivo JSON
        json_path = os.path.join(image_dir, filename)
        
        # Carregar o conteúdo do arquivo JSON
        with open(json_path, 'r') as file:
            try:
                json_data = json.load(file)
            except json.JSONDecodeError as e:
                print(f"Erro ao carregar JSON {json_path}: {e}")
                continue
        
        # Verificar se o sample_id está presente no JSON
        sample_id = str(json_data.get('sample_id', None))
        if sample_id is None:
            print(f"ID da amostra ausente no JSON: {json_path}")
            continue
        
        # Construir o caminho completo do arquivo de imagem
        image_filename = f"{sample_id}.jpg"
        image_path = os.path.join(image_dir, image_filename)
        
        # Verificar se o arquivo de imagem existe
        if not os.path.exists(image_path):
            print(f"Arquivo de imagem não encontrado: {image_path}")
            continue
        
        # Adicionar os dados à lista apropriada
        data_tuple = (image_path, json_data)
        if sample_id in train_ids:
            train_data.append(data_tuple)
            print(f"Imagem de treinamento encontrada e carregada: {image_path}")
        elif sample_id in val_ids:
            val_data.append(data_tuple)
            print(f"Imagem de validação encontrada e carregada: {image_path}")

Imagem de treinamento encontrada e carregada: C:\Users\bruna\OneDrive - Universidade do Minho\Tese Mestrado em Bioinformática\AGAR_dataset\AGAR_dataset\dataset\12994.jpg
Imagem de treinamento encontrada e carregada: C:\Users\bruna\OneDrive - Universidade do Minho\Tese Mestrado em Bioinformática\AGAR_dataset\AGAR_dataset\dataset\12995.jpg
Imagem de treinamento encontrada e carregada: C:\Users\bruna\OneDrive - Universidade do Minho\Tese Mestrado em Bioinformática\AGAR_dataset\AGAR_dataset\dataset\12996.jpg
Imagem de validação encontrada e carregada: C:\Users\bruna\OneDrive - Universidade do Minho\Tese Mestrado em Bioinformática\AGAR_dataset\AGAR_dataset\dataset\12997.jpg
Imagem de treinamento encontrada e carregada: C:\Users\bruna\OneDrive - Universidade do Minho\Tese Mestrado em Bioinformática\AGAR_dataset\AGAR_dataset\dataset\12998.jpg
Imagem de treinamento encontrada e carregada: C:\Users\bruna\OneDrive - Universidade do Minho\Tese Mestrado em Bioinformática\AGAR_dataset\AGAR_dataset\

In [138]:
# Verificar o número de imagens carregadas
print(f"Número de imagens de treinamento carregadas: {len(train_data)}")
print(f"Número de imagens de validação carregadas: {len(val_data)}")

Número de imagens de treinamento carregadas: 3318
Número de imagens de validação carregadas: 1106


In [145]:
# Separar as imagens e os rótulos (labels) para treinamento
train_images = [data[0] for data in train_data]
train_labels = [data[1]['classes'][0] for data in train_data]

In [146]:
# Separar as imagens e os rótulos (labels) para validação
val_images = [data[0] for data in val_data]
val_labels = [data[1]['classes'][0] for data in val_data]

In [147]:
# Verificar se as listas estão vazias
if len(train_images) == 0:
    print("Nenhuma imagem de treinamento foi carregada.")
if len(val_images) == 0:
    print("Nenhuma imagem de validação foi carregada.")

In [148]:
# Converter as imagens em tensores e normalizar, se necessário
train_images = [load_image(image_path, target_size=(224, 224)) for image_path in train_images]
val_images = [load_image(image_path, target_size=(224, 224)) for image_path in val_images]

In [149]:
# Verificar o shape das imagens e dos rótulos antes da codificação
print("Shape de train_images antes da codificação:", np.array(train_images).shape)
print("Shape de train_labels antes da codificação:", np.array(train_labels).shape)
print("Shape de val_images antes da codificação:", np.array(val_images).shape)
print("Shape de val_labels antes da codificação:", np.array(val_labels).shape)

Shape de train_images antes da codificação: (3318, 224, 224, 3)
Shape de train_labels antes da codificação: (3318,)
Shape de val_images antes da codificação: (1106, 224, 224, 3)
Shape de val_labels antes da codificação: (1106,)


In [150]:
# Converter os rótulos em formato adequado (numérico), se necessário
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
val_labels = label_encoder.transform(val_labels)

In [151]:
# Supondo que labels seja uma lista contendo os rótulos de suas imagens
#labels = ["0", "1", "2", "3", "4", "6"]

# Contar classes únicas
#num_classes = len(set(labels))
#print("Número de classes:", num_classes)

Número de classes: 6


In [158]:
#Definir a arquitetura da CNN
num_classes = len(set(train_labels))  # Número de classes
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

In [159]:
# Compilar o modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [161]:
# Dividir os dados em conjuntos de treinamento e validação
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

# Verificar os shapes dos dados após a divisão
print("Shape de train_images após a divisão:", np.array(train_images).shape)
print("Shape de train_labels após a divisão:", np.array(train_labels).shape)
print("Shape de val_images após a divisão:", np.array(val_images).shape)
print("Shape de val_labels após a divisão:", np.array(val_labels).shape)

Shape de train_images após a divisão: (1698, 224, 224, 3)
Shape de train_labels após a divisão: (1698,)
Shape de val_images após a divisão: (425, 224, 224, 3)
Shape de val_labels após a divisão: (425,)


In [168]:
# Compilar o modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Função para gerar batches de dados
def data_generator(image_paths, labels, batch_size, target_size=(224, 224)):
    datagen = ImageDataGenerator()
    while True:
        batch_images = []
        batch_labels = []
        for start in range(0, len(image_paths), batch_size):
            end = min(start + batch_size, len(image_paths))
            for image_path in image_paths[start:end]:
                img = load_image(image_path, target_size)
                if img is not None:
                    batch_images.append(img)
                    batch_labels.append(labels[image_paths.index(image_path)])
            yield np.array(batch_images), np.array(batch_labels)

batch_size = 32  # Defina o tamanho do batch

In [169]:
# Treinar o modelo usando o gerador de dados
epochs = 10  # Definir o número de épocas
train_generator = data_generator(train_images, train_labels, batch_size)
val_generator = data_generator(val_images, val_labels, batch_size)
steps_per_epoch = len(train_images) // batch_size
validation_steps = len(val_images) // batch_size

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=validation_steps
)

Erro ao carregar a imagem [[[0.49019608 0.47843137 0.33333333]
  [0.51764706 0.50980392 0.35686275]
  [0.54117647 0.5372549  0.37647059]
  ...
  [0.6745098  0.60784314 0.48235294]
  [0.67058824 0.59607843 0.46666667]
  [0.66666667 0.59215686 0.45882353]]

 [[0.50588235 0.49411765 0.34901961]
  [0.51764706 0.50980392 0.35686275]
  [0.51764706 0.50196078 0.35686275]
  ...
  [0.6745098  0.61960784 0.48627451]
  [0.66666667 0.60392157 0.46666667]
  [0.65882353 0.58823529 0.45490196]]

 [[0.52941176 0.5254902  0.37647059]
  [0.51764706 0.50980392 0.36078431]
  [0.52941176 0.51372549 0.36078431]
  ...
  [0.64313725 0.56862745 0.44705882]
  [0.65098039 0.58431373 0.45490196]
  [0.65098039 0.58823529 0.45882353]]

 ...

 [[0.78039216 0.70980392 0.49803922]
  [0.74901961 0.67843137 0.49803922]
  [0.78039216 0.69411765 0.49803922]
  ...
  [0.79607843 0.6627451  0.50196078]
  [0.78039216 0.64705882 0.48235294]
  [0.77647059 0.63921569 0.48627451]]

 [[0.78039216 0.70196078 0.48627451]
  [0.784313

ValueError: in user code:

    File "C:\Users\bruna\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\bruna\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\bruna\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\bruna\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1126, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\bruna\anaconda3\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\bruna\anaconda3\Lib\site-packages\keras\src\engine\input_spec.py", line 253, in assert_input_compatibility
        raise ValueError(



In [170]:
# Avaliar o modelo no conjunto de validação
val_loss, val_accuracy = model.evaluate(val_generator, steps=validation_steps)
print(f"Perda na validação: {val_loss}")
print(f"Acurácia na validação: {val_accuracy}")

# Salvar o modelo treinado
#model.save('cnn_model.h5')

Erro ao carregar a imagem [[[0.42352941 0.44705882 0.30588235]
  [0.44705882 0.4745098  0.33333333]
  [0.4745098  0.50588235 0.36470588]
  ...
  [0.60392157 0.56470588 0.46666667]
  [0.61176471 0.55294118 0.4627451 ]
  [0.6        0.54117647 0.45490196]]

 [[0.43529412 0.45882353 0.31764706]
  [0.45098039 0.4745098  0.33333333]
  [0.44313725 0.46666667 0.32941176]
  ...
  [0.60784314 0.58039216 0.47843137]
  [0.59607843 0.55686275 0.4627451 ]
  [0.58823529 0.53333333 0.44313725]]

 [[0.47058824 0.49411765 0.35294118]
  [0.45098039 0.4745098  0.32941176]
  [0.45098039 0.4745098  0.33333333]
  ...
  [0.56862745 0.52156863 0.44313725]
  [0.58431373 0.5372549  0.45098039]
  [0.58039216 0.53333333 0.44705882]]

 ...

 [[0.69411765 0.66666667 0.47843137]
  [0.65490196 0.63137255 0.4627451 ]
  [0.68235294 0.65098039 0.4745098 ]
  ...
  [0.67058824 0.58823529 0.4745098 ]
  [0.65490196 0.56862745 0.44705882]
  [0.65098039 0.55686275 0.45098039]]

 [[0.70196078 0.66666667 0.47058824]
  [0.690196

ValueError: in user code:

    File "C:\Users\bruna\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 2042, in test_function  *
        return step_function(self, iterator)
    File "C:\Users\bruna\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 2025, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\bruna\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 2013, in run_step  **
        outputs = model.test_step(data)
    File "C:\Users\bruna\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1893, in test_step
        y_pred = self(x, training=False)
    File "C:\Users\bruna\anaconda3\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\bruna\anaconda3\Lib\site-packages\keras\src\engine\input_spec.py", line 253, in assert_input_compatibility
        raise ValueError(

  